In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "../data/processed/"
df = pd.read_csv(DATA_PATH + "m5_ts_subset.csv")

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["id", "date"]).reset_index(drop=True)

df.head()

,id,item_id,dept_id,store_id,state_id,d,sales,date,weekday,wday,month,year,event_name_1
0,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,d_1,107,2011-01-29,Saturday,1,1,2011,NaN
1,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,d_2,182,2011-01-30,Sunday,2,1,2011,NaN
2,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,d_3,47,2011-01-31,Monday,3,1,2011,NaN
3,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,d_4,47,2011-02-01,Tuesday,4,2,2011,NaN
4,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,d_5,62,2011-02-02,Wednesday,5,2,2011,NaN


In [2]:
df["dayofweek"] = df["date"].dt.dayofweek
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
df["month"] = df["date"].dt.month
df["year"] = df["date"].dt.year
df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)

In [3]:
LAG_DAYS = [1, 7, 14, 28]

for lag in LAG_DAYS:
    df[f"lag_{lag}"] = (
        df
        .groupby("id")["sales"]
        .shift(lag)
    )

In [4]:
ROLLING_WINDOWS = [7, 14]

for window in ROLLING_WINDOWS:
    df[f"rolling_mean_{window}"] = (
        df
        .groupby("id")["sales"]
        .shift(1)
        .rolling(window)
        .mean()
    )

    df[f"rolling_std_{window}"] = (
        df
        .groupby("id")["sales"]
        .shift(1)
        .rolling(window)
        .std()
    )

In [5]:
df["has_event"] = df["event_name_1"].notna().astype(int)

In [6]:
feature_cols = [
    "sales",
    "dayofweek",
    "weekofyear",
    "month",
    "is_weekend",
    "has_event",
] + \
    [f"lag_{l}" for l in LAG_DAYS] + \
    [f"rolling_mean_{w}" for w in ROLLING_WINDOWS] + \
    [f"rolling_std_{w}" for w in ROLLING_WINDOWS]

df_fe = df.dropna().reset_index(drop=True)

print("Before:", df.shape)
print("After :", df_fe.shape)

Before: (9565, 25)
After : (755, 25)


In [7]:
df_fe[feature_cols].head()

,sales,dayofweek,weekofyear,month,is_weekend,has_event,lag_1,lag_7,lag_14,lag_28,rolling_mean_7,rolling_mean_14,rolling_std_7,rolling_std_14
0,0,2,10,3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,11,3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,3,11,3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,6,11,3,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,6,16,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
OUTPUT_PATH = "../data/processed/"
df_fe.to_csv(OUTPUT_PATH + "m5_features.csv", index=False)

print("Feature-engineered dataset saved")

Feature-engineered dataset saved
